
# Welcome to StableTuner, Let's get started!
#### This notebook will guide you through the setup process.


__[Join the ST Discord for support, chat and fun times :)](https://discord.gg/DahNECrBUZ)__

#### Start by uploading your payload.zip file (just drag and drop it to the file area) and run this cell as it gets uploaded.

In [ ]:
#Much thanks to IndustriaDitat and entmike for helping making ST linux compatible!
from IPython.display import clear_output
from subprocess import getoutput
installed_xformers = False
GPU_CardName = getoutput('nvidia-smi --query-gpu=name --format=csv,noheader')

%pip install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url "https://download.pytorch.org/whl/cu116"
%pip install -U --pre triton
%pip install ninja bitsandbytes
if '3090' in GPU_CardName:
    %pip install https://huggingface.co/industriaditat/xformers_precompiles/resolve/main/RTX3090-xf14-cu116-py38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
    installed_xformers = True
if 'A5000' in GPU_CardName:
    %pip install https://huggingface.co/industriaditat/xformers_precompiles/resolve/main/A5000-xf14-cu116-py38/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64.whl
    installed_xformers = True
if 'T4' in GPU_CardName:
    %pip install https://huggingface.co/r4ziel/xformers_pre_built/resolve/main/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64_t4.whl
    installed_xformers = True
if 'A100' in GPU_CardName:
    %pip install https://huggingface.co/industriaditat/xformers_precompiles/resolve/main/A100_13dev/xformers-0.0.13.dev0-py3-none-any.whl
    installed_xformers = True
if 'V100' in GPU_CardName:
    %pip install https://huggingface.co/industriaditat/xformers_precompiles/resolve/main/V100_13dev/xformers-0.0.13.dev0-py3-none-any.whl
    installed_xformers = True
if installed_xformers == False:
    clear_output()
    print("No precompiled xformers found for your GPU. Please wait while we compile xformers for your GPU, this might take a 10 minutes.")
    %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers
%pip install git+https://github.com/huggingface/diffusers.git@0ca1724#egg=diffusers --force-reinstall
clear_output()
print("Done!")

#### Upload finished?, time to run this next cell!

In [ ]:
!unzip -o -q payload.zip
%pip install -r requirements.txt
clear_output()
print("Done!")

#### Looks like you're done installing, let's get started!



In [ ]:
!accelerate launch --mixed_precision="fp16" scripts/trainer.py --disable_cudnn_benchmark --use_text_files_as_captions --sample_step_interval=500 --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1-base"  --pretrained_vae_name_or_path=""  --output_dir="output/new_model"  --seed=3434554  --resolution=512  --train_batch_size=24  --num_train_epochs=100  --mixed_precision="fp16" --use_bucketing --use_8bit_adam --gradient_checkpointing --gradient_accumulation_steps=1 --learning_rate=3e-6 --lr_warmup_steps=0 --lr_scheduler="constant" --save_latents_cache --train_text_encoder --use_image_names_as_captions --concepts_list="stabletune_concept_list.json" --num_class_images=200 --save_every_n_epoch=5 --n_save_sample=1 --sample_height=512 --sample_width=512 --dataset_repeats=1 --sample_on_training_start